<a href="https://colab.research.google.com/github/kevin-salazar/computacionGrafica/blob/master/3Dstereo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cargar datos para calibrar camara

In [28]:
from google.colab import drive
drive.mount('/content/drive')
!ls '/content/drive/My Drive/3DReconstruction/tableroFotos'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
20191218_170008.jpg  20191218_170310.jpg  20191218_170701.jpg
20191218_170016.jpg  20191218_170316.jpg  20191218_170710.jpg
20191218_170021.jpg  20191218_170321.jpg  20191218_170721.jpg
20191218_170033.jpg  20191218_170326.jpg  20191218_170728.jpg
20191218_170044.jpg  20191218_170405.jpg  20191218_170748.jpg
20191218_170048.jpg  20191218_170409.jpg  20191218_170753.jpg
20191218_170052.jpg  20191218_170415.jpg  20191218_170757.jpg
20191218_170057.jpg  20191218_170422.jpg  20191218_170802.jpg
20191218_170101.jpg  20191218_170428.jpg  20191218_170807.jpg
20191218_170115.jpg  20191218_170436.jpg  20191218_170811.jpg
20191218_170122.jpg  20191218_170437.jpg  20191218_170817.jpg
20191218_170135.jpg  20191218_170443.jpg  20191218_170822.jpg
20191218_170140.jpg  20191218_170454.jpg  20191218_170829.jpg
20191218_170145.jpg  20191218_170458.jpg  20191218_170834.jpg
201

## Camera Calibration


In [0]:
PATH="/content/drive/My Drive/3DReconstruction/"

In [0]:

import cv2
import numpy as np 
import glob
from tqdm import tqdm
import PIL.ExifTags
import PIL.Image

#============================================
# Camera calibration
#============================================

#Define size of chessboard target. 

chessboard_size = (7,5)

#Define arrays to save detected points
obj_points = [] #3D points in real world space 
img_points = [] #3D points in image plane

#Prepare grid and points to display

objp = np.zeros((np.prod(chessboard_size),3),dtype=np.float32)


objp[:,:2] = np.mgrid[0:chessboard_size[0], 0:chessboard_size[1]].T.reshape(-1,2)

#read images

calibration_paths = glob.glob(PATH+'tableroFotos/*')

#Iterate over images to find intrinsic matrix
for image_path in tqdm(calibration_paths):

	#Load image
	image = cv2.imread(image_path)
	gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
	print("Image loaded, Analizying...")
	#find chessboard corners
	ret,corners = cv2.findChessboardCorners(gray_image, chessboard_size, None)

	if ret == True:
		print("Chessboard detected!")
		print(image_path)
		#define criteria for subpixel accuracy
		criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
		#refine corner location (to subpixel accuracy) based on criteria.
		cv2.cornerSubPix(gray_image, corners, (5,5), (-1,-1), criteria)
		obj_points.append(objp)
		img_points.append(corners)

#Calibrate camera
ret, K, dist, rvecs, tvecs = cv2.calibrateCamera(obj_points, img_points,gray_image.shape[::-1], None, None)

#Save parameters into numpy file
np.save(PATH+'parametrosCamara/ret', ret)
np.save(PATH+'parametrosCamara/K', K)
np.save(PATH+'parametrosCamara/dist', dist)
np.save(PATH+'parametrosCamara/rvecs', rvecs)
np.save(PATH+'parametrosCamara/tvecs', tvecs)

#Get exif data in order to get focal length. 
exif_img = PIL.Image.open(calibration_paths[0])

exif_data = {
	PIL.ExifTags.TAGS[k]:v
	for k, v in exif_img._getexif().items()
	if k in PIL.ExifTags.TAGS}

#Get focal length in tuple form
focal_length_exif = exif_data['FocalLength']

#Get focal length in decimal form
focal_length = focal_length_exif[0]/focal_length_exif[1]

#Save focal length
np.save(PATH+'parametrosCamara/FocalLength', focal_length)

#Calculate projection error. 
mean_error = 0
for i in range(len(obj_points)):
	img_points2, _ = cv2.projectPoints(obj_points[i],rvecs[i],tvecs[i], K, dist)
	error = cv2.norm(img_points[i], img_points2, cv2.NORM_L2)/len(img_points2)
	mean_error += error

total_error = mean_error/len(obj_points)
print (total_error)

  0%|          | 0/76 [00:00<?, ?it/s]

Image loaded, Analizying...


  1%|▏         | 1/76 [01:18<1:38:08, 78.52s/it]

Image loaded, Analizying...


  3%|▎         | 2/76 [02:32<1:35:03, 77.08s/it]

...

Image loaded, Analizying...


 39%|███▉      | 30/76 [35:15<44:52, 58.53s/it]

Image loaded, Analizying...


 41%|████      | 31/76 [36:25<46:24, 61.87s/it]

Image loaded, Analizying...


## Epipolar Geometry


In [0]:
'''
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt
img1 = cv.imread(PATH+'reconstruct_this/image1.png',0)  #queryimage # left image
img2 = cv.imread(PATH+'reconstruct_this/image1.png',0) #trainimage # right image
#sift = cv.SIFT()
sift = cv2.xfeatures2d.SURF_create()

# find the keypoints and descriptors with SIFT
kp1, des1 = sift.detectAndCompute(img1,None)
kp2, des2 = sift.detectAndCompute(img2,None)
# FLANN parameters
FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks=50)
flann = cv.FlannBasedMatcher(index_params,search_params)
matches = flann.knnMatch(des1,des2,k=2)
good = []
pts1 = []
pts2 = []
# ratio test as per Lowe's paper
for i,(m,n) in enumerate(matches):
    if m.distance < 0.8*n.distance:
        good.append(m)
        pts2.append(kp2[m.trainIdx].pt)
        pts1.append(kp1[m.queryIdx].pt)
'''

![texto alternativo](https://docs.opencv.org/3.4.4/epipolar.jpg)


![texto alternativo](https://docs.opencv.org/3.4.4/epiresult.jpg)


## 3D Reconstruction

In [0]:
import cv2
import numpy as np 
import glob
from tqdm import tqdm
import PIL.ExifTags
import PIL.Image
from matplotlib import pyplot as plt 

#=====================================
# Function declarations
#=====================================

#Function to create point cloud file
def create_output(vertices, colors, filename):
	colors = colors.reshape(-1,3)
	vertices = np.hstack([vertices.reshape(-1,3),colors])

	ply_header = '''ply
		format ascii 1.0
		element vertex %(vert_num)d
		property float x
		property float y
		property float z
		property uchar red
		property uchar green
		property uchar blue
		end_header
		'''
	with open(filename, 'w') as f:
		f.write(ply_header %dict(vert_num=len(vertices)))
		np.savetxt(f,vertices,'%f %f %f %d %d %d')

#Function that Downsamples image x number (reduce_factor) of times. 
def downsample_image(image, reduce_factor):
	for i in range(0,reduce_factor):
		#Check if image is color or grayscale
		if len(image.shape) > 2:
			row,col = image.shape[:2]
		else:
			row,col = image.shape

		image = cv2.pyrDown(image, dstsize= (col//2, row // 2))
	return image


#=========================================================
# Stereo 3D reconstruction 
#=========================================================

#Load camera parameters
ret = np.load(PATH+'parametrosCamara/ret.npy')
K = np.load(PATH+'parametrosCamara/K.npy')
dist = np.load(PATH+'parametrosCamara/dist.npy')

#Specify image paths
img_path1 = PATH+'stereoFotos/sillaL.jpg'
img_path2 = PATH+'stereoFotos/sillaR.jpg'

#Load pictures
img_1 = cv2.imread(img_path1)
img_2 = cv2.imread(img_path2)

#Get height and width. Note: It assumes that both pictures are the same size. They HAVE to be same size and height. 
h,w = img_2.shape[:2]

#Get optimal camera matrix for better undistortion 
new_camera_matrix, roi = cv2.getOptimalNewCameraMatrix(K,dist,(w,h),1,(w,h))

#Undistort images
img_1_undistorted = cv2.undistort(img_1, K, dist, None, new_camera_matrix)
img_2_undistorted = cv2.undistort(img_2, K, dist, None, new_camera_matrix)

#Downsample each image 3 times (because they're too big)
img_1_downsampled = downsample_image(img_1_undistorted,3)
img_2_downsampled = downsample_image(img_2_undistorted,3)

#cv2.imwrite('undistorted_left.jpg', img_1_downsampled)
#cv2.imwrite('undistorted_right.jpg', img_2_downsampled)


#Set disparity parameters
#Note: disparity range is tuned according to specific parameters obtained through trial and error. 
win_size = 5
min_disp = -1
max_disp = 63 #min_disp * 9
num_disp = max_disp - min_disp # Needs to be divisible by 16

#Create Block matching object. 
stereo = cv2.StereoSGBM_create(minDisparity= min_disp,
	numDisparities = num_disp,
	blockSize = 5,
	uniquenessRatio = 5,
	speckleWindowSize = 5,
	speckleRange = 5,
	disp12MaxDiff = 2,
	P1 = 8*3*win_size**2,#8*3*win_size**2,
	P2 =32*3*win_size**2) #32*3*win_size**2)

#Compute disparity map
print ("\nComputing the disparity  map...")
disparity_map = stereo.compute(img_1_downsampled, img_2_downsampled)

#Show disparity map before generating 3D cloud to verify that point cloud will be usable. 
plt.imshow(disparity_map,'gray')
plt.show()

#Generate  point cloud. 
print ("\nGenerating the 3D map...")

#Get new downsampled width and height 
h,w = img_2_downsampled.shape[:2]

#Load focal length. 
focal_length = np.load(PATH+'parametrosCamara/FocalLength.npy')

#Perspective transformation matrix
#This transformation matrix is from the openCV documentation, didn't seem to work for me. 
Q = np.float32([[1,0,0,-w/2.0],
				[0,-1,0,h/2.0],
				[0,0,0,-focal_length],
				[0,0,1,0]])

#This transformation matrix is derived from Prof. Didier Stricker's power point presentation on computer vision. 
#Link : https://ags.cs.uni-kl.de/fileadmin/inf_ags/3dcv-ws14-15/3DCV_lec01_camera.pdf
Q2 = np.float32([[1,0,0,0],
				[0,-1,0,0],
				[0,0,focal_length*0.05,0], #Focal length multiplication obtained experimentally. 
				[0,0,0,1]])

#Reproject points into 3D
points_3D = cv2.reprojectImageTo3D(disparity_map, Q2)
#Get color points
colors = cv2.cvtColor(img_1_downsampled, cv2.COLOR_BGR2RGB)

#Get rid of points with value 0 (i.e no depth)
mask_map = disparity_map > disparity_map.min()

#Mask colors and points. 
output_points = points_3D[mask_map]
output_colors = colors[mask_map]

#Define name for output file
output_file = 'reconstructed.ply'

#Generate point cloud 
print("\n Creating the output file... \n")
create_output(output_points, output_colors, output_file)